In [1]:
import sys
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
sys.path.append(".")
# os.environ["MAX_JOBS"] = "100"

import torch
torch.set_printoptions(linewidth=200)
torch.cuda.is_available()

True

In [2]:
from src.vlstm_fw_v0.interface import vlstm_fw_torch
from src.vlstm_fw_v0.interface import vlstm_fw_cuda

INCLUDE: ['/home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include', '/home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include/torch/csrc/api/include', '/home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include/TH', '/home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include/THC', '/home/max/miniconda3/envs/xlstmpt21cu118/include']
/home/max/myrepos/vlstm_cuda/src
/home/max/cpplibs/libtorch/lib:/usr/local/cuda-12.3/lib64:
/home/max/miniconda3/envs/xlstmpt21cu118/lib


Using /home/max/.cache/torch_extensions/py311_cu118 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/max/.cache/torch_extensions/py311_cu118/vlstm_fw_v0/build.ninja...
Building extension module vlstm_fw_v0...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


[1/2] /home/max/miniconda3/envs/xlstmpt21cu118/bin/nvcc  -ccbin /home/max/miniconda3/envs/xlstmpt21cu118/bin/x86_64-conda-linux-gnu-cc -DTORCH_EXTENSION_NAME=vlstm_fw_v0 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include -isystem /home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include/torch/csrc/api/include -isystem /home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include/TH -isystem /home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include/THC -isystem /home/max/miniconda3/envs/xlstmpt21cu118/include -isystem /home/max/miniconda3/envs/xlstmpt21cu118/include/python3.11 -D_GLIBCXX_USE_CXX11_ABI=0 -D__CUDA_NO_HALF_OPERATORS__ -D__CUDA_NO_HALF_CONVERSIONS__ -D__CUDA_NO_BFLOAT16_CONVERSIONS__ -D__CUDA_NO_HALF2_OPERATORS__ --expt-relaxe

Loading extension module vlstm_fw_v0...


## CUDA vLSTM forward

In [3]:
S = 8 # seq len
B = 1 # batch size
NH = 1 # num heads
DH = 4 # dim per head
DTYPE = torch.float32
DEVICE = torch.device("cuda:0")

In [4]:
# create qkv, inputgates, forgetgates 
torch.manual_seed(0)
qs = torch.arange((B*NH*S*DH), device=DEVICE, dtype=DTYPE).reshape((B, NH, S, DH)) / 10.
ks = torch.ones((B, NH, S, DH), device=DEVICE, dtype=DTYPE) / 100.
vs = torch.ones((B, NH, S, DH), device=DEVICE, dtype=DTYPE) / 100.
# igs = (1. + torch.arange((B * NH * S), device=DEVICE, dtype=DTYPE)).reshape(B, NH, S, 1) / 100.
igs = torch.ones((B, NH, S, 1), device=DEVICE, dtype=DTYPE) #/ 10.
fgs = 0.01 * torch.ones((B, NH, S, 1), device=DEVICE, dtype=DTYPE)
qs, qs.shape, len(qs.view(-1))

(tensor([[[[0.0000, 0.1000, 0.2000, 0.3000],
           [0.4000, 0.5000, 0.6000, 0.7000],
           [0.8000, 0.9000, 1.0000, 1.1000],
           [1.2000, 1.3000, 1.4000, 1.5000],
           [1.6000, 1.7000, 1.8000, 1.9000],
           [2.0000, 2.1000, 2.2000, 2.3000],
           [2.4000, 2.5000, 2.6000, 2.7000],
           [2.8000, 2.9000, 3.0000, 3.1000]]]], device='cuda:0'),
 torch.Size([1, 1, 8, 4]),
 32)

In [5]:
# pytorch version
rs, log_fgm, igm = vlstm_fw_torch(
    queries=qs,
    keys=ks,
    values=vs,
    igate_preact=igs,
    fgate_preact=fgs,
)
rs, rs.shape

(tensor([[[[8.1548e-05, 8.1548e-05, 8.1548e-05, 8.1548e-05],
           [4.4926e-04, 4.4926e-04, 4.4926e-04, 4.4926e-04],
           [9.0641e-04, 9.0641e-04, 9.0641e-04, 9.0641e-04],
           [1.3812e-03, 1.3812e-03, 1.3812e-03, 1.3812e-03],
           [1.8511e-03, 1.8511e-03, 1.8511e-03, 1.8511e-03],
           [2.3116e-03, 2.3116e-03, 2.3116e-03, 2.3116e-03],
           [2.7640e-03, 2.7640e-03, 2.7640e-03, 2.7640e-03],
           [3.2106e-03, 3.2106e-03, 3.2106e-03, 3.2106e-03]]]], device='cuda:0'),
 torch.Size([1, 1, 8, 4]))

In [6]:
# pytorch version QKV product (To test if this is still correct after changes to the code.)
# at some point we have to compare to the vlstm_fw_torch version.
rs = qs @ ks.transpose(-1, -2) @ vs
rs, rs.shape

(tensor([[[[0.0005, 0.0005, 0.0005, 0.0005],
           [0.0018, 0.0018, 0.0018, 0.0018],
           [0.0030, 0.0030, 0.0030, 0.0030],
           [0.0043, 0.0043, 0.0043, 0.0043],
           [0.0056, 0.0056, 0.0056, 0.0056],
           [0.0069, 0.0069, 0.0069, 0.0069],
           [0.0082, 0.0082, 0.0082, 0.0082],
           [0.0094, 0.0094, 0.0094, 0.0094]]]], device='cuda:0'),
 torch.Size([1, 1, 8, 4]))

In [7]:
rs = (qs @ ks.transpose(-1, -2) * torch.tril(torch.ones((B, NH, S, S))).to(device=DEVICE, dtype=DTYPE)) @ vs
rs, rs.shape

(tensor([[[[6.0000e-05, 6.0000e-05, 6.0000e-05, 6.0000e-05],
           [4.4000e-04, 4.4000e-04, 4.4000e-04, 4.4000e-04],
           [1.1400e-03, 1.1400e-03, 1.1400e-03, 1.1400e-03],
           [2.1600e-03, 2.1600e-03, 2.1600e-03, 2.1600e-03],
           [3.5000e-03, 3.5000e-03, 3.5000e-03, 3.5000e-03],
           [5.1600e-03, 5.1600e-03, 5.1600e-03, 5.1600e-03],
           [7.1400e-03, 7.1400e-03, 7.1400e-03, 7.1400e-03],
           [9.4400e-03, 9.4400e-03, 9.4400e-03, 9.4400e-03]]]], device='cuda:0'),
 torch.Size([1, 1, 8, 4]))

In [8]:
# cuda kernel
rs = vlstm_fw_cuda(mat_Q=qs, mat_K=ks, mat_V=vs)
rs, rs.shape

before kernel dispatch - float32!
B: 1, NH: 1, S: 8, DH: 4
blocksxy: 1-2, threadsxy: 4-4, shared_mem in bytes: 2560
In Kernel: gdim.x: 1, gdim.y: 2, gdim.z: 1, bdim.x: 4, bdim.y: 4
In Kernel: QtileDim: 8, KVtileDim: 8, TblockDim:4
qTileIdx=0: qTileEnd: 1, qTileGridXYGlobalMemIdx: 0, qTileBlockGlobalMemIdx: 0
qWarpTileYIdx=0: qWarpTileYEnd: 2, qWarpTileXEnd: 1
qWarpTileYIdx=1: qWarpTileYEnd: 2, qWarpTileXEnd: 1
kvWarpTileYIdx=0: kvWarpTileYEnd: 2, kvWarpTileXEnd: 1
kvWarpTileYIdx=1: kvWarpTileYEnd: 2, kvWarpTileXEnd: 1


(tensor([[[[6.0000e-05, 6.0000e-05, 6.0000e-05, 6.0000e-05],
           [4.4000e-04, 4.4000e-04, 4.4000e-04, 4.4000e-04],
           [1.1400e-03, 1.1400e-03, 1.1400e-03, 1.1400e-03],
           [2.1600e-03, 2.1600e-03, 2.1600e-03, 2.1600e-03],
           [3.5000e-03, 3.5000e-03, 3.5000e-03, 3.5000e-03],
           [5.1600e-03, 5.1600e-03, 5.1600e-03, 5.1600e-03],
           [7.1400e-03, 7.1400e-03, 7.1400e-03, 7.1400e-03],
           [9.4400e-03, 9.4400e-03, 9.4400e-03, 9.4400e-03]]]], device='cuda:0'),
 torch.Size([1, 1, 8, 4]))

In [9]:
qs @ ks.transpose(-1, -2)

tensor([[[[0.0060, 0.0060, 0.0060, 0.0060, 0.0060, 0.0060, 0.0060, 0.0060],
          [0.0220, 0.0220, 0.0220, 0.0220, 0.0220, 0.0220, 0.0220, 0.0220],
          [0.0380, 0.0380, 0.0380, 0.0380, 0.0380, 0.0380, 0.0380, 0.0380],
          [0.0540, 0.0540, 0.0540, 0.0540, 0.0540, 0.0540, 0.0540, 0.0540],
          [0.0700, 0.0700, 0.0700, 0.0700, 0.0700, 0.0700, 0.0700, 0.0700],
          [0.0860, 0.0860, 0.0860, 0.0860, 0.0860, 0.0860, 0.0860, 0.0860],
          [0.1020, 0.1020, 0.1020, 0.1020, 0.1020, 0.1020, 0.1020, 0.1020],
          [0.1180, 0.1180, 0.1180, 0.1180, 0.1180, 0.1180, 0.1180, 0.1180]]]], device='cuda:0')